In [1]:
import collections

from Bio import pairwise2
from Bio.Seq import Seq
import matplotlib.pyplot as plt
#import networkx as nx
import numpy as np
import pandas as pd

from data import AllBooks, Book
from utils import most_frequent

#import bible_datasets

#from tf.app import use
#MT = use('etcbc/bhsa')
#Fmt, Tmt, Lmt = MT.api.F, MT.api.T, MT.api.L

#SP = use('dt-ucph/sp', version=sp_version)
#Fsp, Tsp, Lsp = SP.api.F, SP.api.T, SP.api.L

#DSS = use('etcbc/dss', version=dss_version)
#Fdss, Tdss, Ldss = DSS.api.F, DSS.api.T, DSS.api.L

from align_config import dss_version, sp_version
from align_functions import make_alignments, collect_matching_words

from tf.app import use

MT = use('etcbc/bhsa')
Fmt, Tmt, Lmt = MT.api.F, MT.api.T, MT.api.L

SP = use('dt-ucph/sp', version=sp_version)
Fsp, Tsp, Lsp = SP.api.F, SP.api.T, SP.api.L

DSS = use('etcbc/dss', version=dss_version)
Fdss, Tdss, Ldss = DSS.api.F, DSS.api.T, DSS.api.L


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,5,79878.40,100
chapter,187,2135.79,100
verse,5841,68.38,100
word,114891,3.48,100
sign,399392,1.00,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


In [2]:
MATRES_FILE = '../data/ptca_qal.csv'

In [3]:
PENTATEUCH_BOOKS = ['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy']
ALL_BOOK_NAMES = [Tmt.sectionFromNode(bo)[0] for bo in Fmt.otype.s('book')]

In [4]:
QSP_SCROLLS = {'1Qisaa', '1QisaaI', '1QisaaII', '2Q3', '4Q13', '4Q20', '2Q7', '4Q27', '1Q4', '2Q12', '4Q37', '4Q38', '4Q38a', '4Q40', '4Q53',
               '4Q57', '2Q13', '4Q78', '4Q80', '4Q82', '4Q128', '4Q129', '4Q134', '4Q135', '4Q136',
                '4Q137', '4Q138', '4Q139', '4Q140', '4Q141', '4Q142', '4Q143', '4Q144', '4Q158', '4Q364',
                '4Q365', '4Q96', '4Q111', '4Q109', '11Q5', '11Q6', '11Q7', '11Q8'}

# Prepare MT and SP texts

Produce dictionary mt_sp_matches which has mt words nodes as keys and matching word numbers from SP as values.

In [5]:
# prepare mt and sp books
MANUSCRIPTS = ['MT', 'SP']
all_books = AllBooks()
for book_name in ALL_BOOK_NAMES:
    book = Book('MT', book_name, Fmt, Tmt, Lmt)
    all_books.data[('MT', book_name)] = book
    
    if book_name in PENTATEUCH_BOOKS:
        book = Book('SP', book_name, Fsp, Tsp, Lsp)
        all_books.data[('SP', book_name)] = book

# Match words

In [7]:
dat = pd.read_csv(MATRES_FILE, sep='\t')

# dat_dss means non-MT, SP is included
dat_dss = dat[~dat.scroll.isin(['MT'])]
scroll_book_combinations = list(set(zip(dat_dss.scroll, dat_dss.book)))
dat_dss.shape

(2029, 35)

In [8]:
for manuscript, book_name in scroll_book_combinations:
    if manuscript == 'MT':
        continue
    elif manuscript == 'SP':
        book = Book(manuscript, book_name, Fsp, Tsp, Lsp)
        all_books.data[(manuscript, book_name)] = book
    else:
        book = Book(manuscript, book_name, Fdss, Tdss, Ldss)
        all_books.data[(manuscript, book_name)] = book

In [18]:
def make_matching_word_dict(book_name, all_books):

    all_match_dicts = {}

    matching_book = book_name
    matching_manuscripts = [scr for scr, bo in all_books.data.keys() if bo == matching_book]
    #print(matching_manuscripts)

    for man1 in matching_manuscripts:
        for man2 in matching_manuscripts:
            if man1 != man2:
            
                man1_verse_texts = all_books.data[(man1, matching_book)].verse_text_dict
                man2_verse_texts = all_books.data[(man2, matching_book)].verse_text_dict

                alignments_dict = make_alignments(man1_verse_texts, man2_verse_texts)
    
                man1_word2char = all_books.data[(man1, matching_book)].word2char
                man2_word2char = all_books.data[(man2, matching_book)].word2char
 
                matching_words_dict = collect_matching_words(alignments_dict, man1_word2char, man2_word2char)

                matches = {}
                for man1_word, man2_list in matching_words_dict.items():
                    man2_word = most_frequent(man2_list)
                    matches[man1_word] = man2_word
                all_match_dicts[((man1, matching_book), (man2, matching_book))] = matches
                
    return all_match_dicts, matching_manuscripts

In [19]:
def read_dataset(file):

    return pd.read_csv(file, sep='\t')

In [20]:
class MaterData:
    def __init__(self, man1, man2, section, lex, mater_val1, mater_val2, tf_id1, tf_id2, g_cons1, g_cons2):
        self.man1 = man1
        self.man2 = man2
        self.section = section
        self.lex = lex
        self.mater_val1 = mater_val1
        self.mater_val2 = mater_val2
        self.tf_id1 = tf_id1
        self.tf_id2 = tf_id2
        self.g_cons1 = g_cons1
        self.g_cons2 = g_cons2

In [21]:
def collect_matching_cases(matching_manuscripts, matching_book, dat):

    all_mater_datasets = {}

    for man in matching_manuscripts:
        mater_data = dat[(dat.book == matching_book) & (dat.scroll == man)]
        all_mater_datasets[(man, matching_book)] = mater_data
        
    return all_mater_datasets

In [22]:
def collect_mater_data(matching_manuscripts, all_match_dicts, all_mater_datasets, matching_book):

    manuscript_mater_match = collections.defaultdict(list)
    manuscripts = set()

    for idx, man in enumerate(matching_manuscripts):
        for idx2, man2 in enumerate(matching_manuscripts):
            if idx < idx2:
            
                matching_ids = all_match_dicts[((man, matching_book), (man2, matching_book))]
                man_data = all_mater_datasets[(man, matching_book)]
                man2_data = all_mater_datasets[(man2, matching_book)]
                for _, row in man_data.iterrows():
                
                    tf_id = row.tf_id
                    lex, typ, has_vl = row.lex, row.type, row.has_vowel_letter
                    g_cons1 = row.g_cons
                    section = (row.book, row.chapter, row.verse)
                    if man == 'SP':
                        tf_id = tf_id - 100000

                    matching_tf_id = matching_ids.get(tf_id, None)
                
                    if not matching_tf_id:
                        continue
                    
                    if man2 == 'SP':
                        matching_tf_id = matching_tf_id + 100000
                
                    man2_row = man2_data[(man2_data.tf_id == matching_tf_id) & (man2_data.lex == lex) & (man2_data.type == typ)]
                    
                    if not man2_row.shape[0]:
                        continue
                    has_vl2 = man2_row.has_vowel_letter.iloc[0]
                    g_cons2 = man2_row.g_cons.iloc[0]
                 
                    mater_data = MaterData(man, man2, section, lex, has_vl, has_vl2, tf_id, matching_tf_id, g_cons1, g_cons2)
                
                    manuscript_mater_match[man].append(mater_data)
                    manuscripts.add(man)
                    manuscripts.add(man2)
                    
    return manuscript_mater_match, manuscripts

In [23]:
def register_similarities_with_mt(manuscripts, mt_ids, manuscript_mater_match):

    mater_value_dict = {0: -1,
                    1: 1}

    mater_match_array = np.zeros((len(manuscripts), len(mt_ids)))

    for dat_object in manuscript_mater_match['MT']:
        
        other_man = dat_object.man2
        mt_tfid = dat_object.tf_id1
        other_man_tfid = dat_object.tf_id2
    
        mt_mater = dat_object.mater_val1
        other_man_mater = dat_object.mater_val2
    
        mt_idx = man2idx['MT']
        other_man_idx = man2idx[other_man]
    
        mt_mater_value = mater_value_dict[mt_mater]
        other_man_mater_value = mater_value_dict[other_man_mater]
    
        mt_tf_id = mt_tf2idx[mt_tfid]
    
        mater_match_array[mt_idx, mt_tf_id] = mt_mater_value
        mater_match_array[other_man_idx, mt_tf_id] = other_man_mater_value
        
    return mater_match_array

In [24]:
def get_parallels(manuscript_mater_match, hif_match_dict):
    for scroll in manuscript_mater_match.keys():
        for dat_object in manuscript_mater_match[scroll]:
            man1 = dat_object.man1
            man2 = dat_object.man2
            if man1 == 'MT' or man2 == 'MT':
            
                hif_match_dict[dat_object.tf_id2] = dat_object.tf_id1
                hif_match_dict[dat_object.tf_id1] = dat_object.tf_id2
        
    return hif_match_dict


In [25]:
def count_parallel_cases(mater_match_array):

    mater_arr = np.zeros((2, mater_match_array.shape[1]))

    for col_idx in range(mater_match_array.shape[1]):
        col = mater_match_array[:, col_idx]
        col_counts = collections.Counter(col)
        with_vowel_count = col_counts.get(1, 0)
        without_vowel_count = col_counts.get(-1, 0)
        mater_arr[0, col_idx] = with_vowel_count
        mater_arr[1, col_idx] = without_vowel_count
        
    return mater_arr

In [26]:
#file_name = '../data/ptca_qal.csv'
#all_bib_books = [Tmt.sectionFromNode(bo)[0] for bo in Fmt.otype.s('book')]
#dat = read_dataset(MATRES_FILE)

hif_match_dict = {}

for book in ALL_BOOK_NAMES:
    print(book)
    all_match_dicts, matching_manuscripts = make_matching_word_dict(book, all_books)
    all_mater_datasets = collect_matching_cases(matching_manuscripts, book, dat)
    manuscript_mater_match, manuscripts = collect_mater_data(matching_manuscripts, all_match_dicts, all_mater_datasets, book)
    
    hif_match_dict = get_parallels(manuscript_mater_match, hif_match_dict)
    print(len(hif_match_dict))
    

Genesis
136
Exodus
211
Leviticus
252
Numbers
283
Deuteronomy
437
Joshua
455
Judges
459
1_Samuel
471
2_Samuel
489
1_Kings
493
2_Kings
493
Isaiah
1466
Jeremiah
1522
Ezekiel
1540
Hosea
1542
Joel
1550
Amos
1576
Obadiah
1583
Jonah
1603
Micah
1649
Nahum
1674
Habakkuk
1682
Zephaniah
1705
Haggai
1705
Zechariah
1713
Malachi
1731
Psalms
1966
Job
1970
Proverbs
1986
Ruth
1990
Song_of_songs
1998
Ecclesiastes
2000
Lamentations
2016
Esther
2016
Daniel
2028
Ezra
2028
Nehemiah
2028
1_Chronicles
2028
2_Chronicles
2028


In [27]:
hif_match_dict

{505611: 191,
 191: 1937930,
 505636: 216,
 216: 1937956,
 505819: 395,
 395: 1940640,
 505953: 530,
 530: 505953,
 506010: 586,
 586: 506010,
 506023: 599,
 599: 506023,
 506039: 616,
 616: 506039,
 506286: 865,
 865: 506286,
 506309: 888,
 888: 506309,
 506343: 921,
 921: 506343,
 506357: 935,
 935: 1938320,
 506670: 1243,
 1243: 506670,
 506684: 1257,
 1257: 506684,
 507120: 1697,
 1697: 507120,
 507125: 1702,
 1702: 1938460,
 507201: 1779,
 1779: 1938541,
 507245: 1821,
 1821: 1938584,
 507255: 1831,
 1831: 507255,
 507327: 1905,
 1905: 507327,
 507335: 1913,
 1913: 507335,
 507350: 1928,
 1928: 507350,
 507378: 1956,
 1956: 507378,
 507434: 2011,
 2011: 507434,
 507446: 2023,
 2023: 507446,
 507457: 2034,
 2034: 507457,
 507459: 2036,
 2036: 507459,
 508609: 3193,
 3193: 508609,
 508732: 3320,
 3320: 508732,
 508867: 3453,
 3453: 508867,
 508885: 3474,
 3474: 508885,
 509291: 3893,
 3893: 509291,
 509531: 4135,
 4135: 509531,
 509603: 4209,
 4209: 509603,
 509645: 4250,
 4250: 509

In [28]:
len(hif_match_dict)

2028

In [29]:
all_scrolls = set(dat_dss.scroll) 
non_qsp_scrolls = all_scrolls.difference(QSP_SCROLLS)
non_qsp_data = dat_dss[dat_dss.scroll.isin(non_qsp_scrolls)]
non_qsp_pent = non_qsp_data[non_qsp_data.book.isin(PENTATEUCH_BOOKS)]
non_qsp_pent.shape

(1020, 35)

In [30]:
n = 0
full_count = collections.defaultdict(int)
scrolls = set()
scroll_counts = collections.defaultdict(int)

sp_mt_dict = collections.defaultdict(lambda: collections.defaultdict(int))

for index, row in non_qsp_data.iterrows():
    tf_id = row.tf_id, 
    scroll = row.scroll, 
    has_vl = row.has_vowel_letter
    book, ch, ve = row.book, row.chapter, row.verse
    g_cons = row.g_cons
    lex = row.lex
    prefix = row.prefix
    #print(tf_id)
    #if lex != 'NBJ>/':
    #    continue
    
    
    
    #if scroll[0] not in {'SP'}: #, '4Q55'}:
    #    continue
    #if book != 'Isaiah':
    #    continue
        
    #if scroll[0] not in {'Mur88'}: #, '4Q55'}:
    #    continue
    #if lex[2] != 'H':
    #    continue
    
    
    
    if tf_id[0] in hif_match_dict:
        corr_id = hif_match_dict[tf_id[0]]
        corr_row = dat[dat.tf_id == corr_id]
        #print(scroll, corr_row.scroll.iloc[0], row.chapter, row.verse, row.lex, g_cons, corr_row.g_cons.iloc[0], 
        #      book, prefix, corr_row.prefix.iloc[0], has_vl, corr_row.has_vowel_letter.iloc[0])
        n += 1
        
        mt_full = corr_row.has_vowel_letter.iloc[0]
        scroll_full = has_vl
        full_count[(scroll_full, mt_full)] += 1
        scrolls.add(scroll)
        scroll_counts[scroll] += 1
        
        # SP MT
        #if scroll[0] == 'SP' and  corr_row.scroll.iloc[0] == 'MT':
        #    sp_mt_dict[lex][(scroll_full, mt_full)] += 1
        #    if lex == 'JCB[':
        #        print(has_vl, corr_row.has_vowel_letter.iloc[0], g_cons, corr_row.g_cons.iloc[0], prefix, book, ch, ve)
        
        if scroll[0] != 'SP' and lex[0] != 'J':
            print(scroll, has_vl, corr_row.has_vowel_letter.iloc[0], g_cons, corr_row.g_cons.iloc[0], prefix, book, ch, ve)
        
    else:
        #print(scroll, book, row.chapter, row.verse, row.lex, row.g_cons)
        if scroll[0] != 'SP' and lex[0] == 'J':
               print(scroll, book, row.chapter, row.verse, row.lex, row.g_cons)
        continue
    
print(n)
print(full_count)
print(len((scrolls)))
print(scroll_counts)

('1Q5',) 0 0 KRT KRT nan Deuteronomy 29 13
('1Q5',) 0 0 PNH PNH nan Deuteronomy 29 17
('1Q5',) 1 0 <WBR <BR nan Deuteronomy 31 3
('1Q5',) 0 0 NF>JM NF>JM H Deuteronomy 31 9
('1Q8',) 0 0 PRFJ PRFJ W Isaiah 19 8
('1Q8',) 0 0 >RGJM >RGJM W Isaiah 19 9
('1Q8',) 1 1 <WLH <WLH WH Isaiah 24 18
('1Q8',) Isaiah 28 16 JSD[ JWSD
('1Q8',) 1 0 <WBR <BR nan Isaiah 29 5
('1Q8',) 0 0 <BDJ <BDJ nan Isaiah 30 24
('1Q8',) 0 0 R<J R<J nan Isaiah 38 12
('1Q8',) 0 0 YRP YRP nan Isaiah 41 7
('1Q8',) 1 1 HWLM HWLM nan Isaiah 41 7
('1Q8',) 0 0 >HBJ >HBJ nan Isaiah 41 8
('1Q8',) 0 0 G>LK G>LK W Isaiah 41 14
('1Q8',) 1 0 BWR>K BR>K nan Isaiah 43 1
('1Q8',) 1 0 GW>LK G>LK nan Isaiah 44 24
('1Q8',) 0 0 <FH <FH nan Isaiah 44 24
('1Q8',) 1 0 RWQ< RQ< nan Isaiah 44 24
('1Q8',) 1 0 QWSMJM QSMJM W Isaiah 44 25
('1Q8',) 1 0 >WMR >MR H Isaiah 44 26
('1Q8',) 1 0 >WMR >MR H Isaiah 44 28
('1Q8',) 0 0 R<J R<J nan Isaiah 44 28
('1Q8',) 1 1 QWR> QWR> H Isaiah 45 3
('1Q8',) 1 0 >WMR >MR H Isaiah 45 10
('1Q8',) 0 0 >TJWT >TJWT H

('Xq3',) 0 0 FN>J FN>J L Deuteronomy 5 9
('Xq3',) 0 0 <FH <FH W Deuteronomy 5 10
('Xq3',) 0 0 >HBJ >HBJ L Deuteronomy 5 10
('Xq3',) 0 0 CMRJ CMRJ WL Deuteronomy 5 10
('Xq3',) 0 0 NTN NTN nan Deuteronomy 5 16
('Xq3',) 0 0 ZBX ZBX nan Exodus 13 15
('Mur4',) 0 0 ZBX ZBX nan Exodus 13 15
('Mur4',) 0 0 NTN NTN nan Deuteronomy 11 17
('Mur88',) 0 0 QR> QR> nan Joel 3 5
('Mur88',) 0 0 GMLJM GMLJM nan Joel 4 4
('Mur88',) 1 1 NWG< NWG< H Amos 9 5
('Mur88',) 1 1 BWNH BWNH H Amos 9 6
('Mur88',) 0 0 QR> QR> H Amos 9 6
('Mur88',) 0 0 >MRJM >MRJM H Amos 9 10
('Mur88',) 0 0 <FH <FH nan Amos 9 12
('Mur88',) 0 0 QYR QYR B Amos 9 13
('Mur88',) 0 0 DRK DRK W Amos 9 13
('Mur88',) 0 0 CKNJ CKNJ nan Obadiah 1 3
('Mur88',) 0 0 >MR >MR nan Obadiah 1 3
('Mur88',) 0 0 BYRJM BYRJM nan Obadiah 1 5
('Mur88',) 0 0 BRX BRX nan Jonah 1 10
('Mur88',) 1 1 HWLK HWLK nan Jonah 1 11
('Mur88',) 0 0 S<R S<R W Jonah 1 11
('Mur88',) 1 1 HWLK HWLK nan Jonah 1 13
('Mur88',) 0 0 S<R S<R W Jonah 1 13
('Mur88',) 0 0 DBR DBR nan Jon

In [ ]:
sp_mt_dict